In [1]:

import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [4]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_1_70per_training_dataset5000_Feb_2024_set21.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.0,-0.385002,0.397897,0.923608,-0.385002,-0.385002,0.923608,-0.385002,0.000000,0.397897,0.000000,0.397897,0.000000,0.397897,1.354520,-0.385002,0.397897,-0.385002,0.923608,0.923608
1,0.0,1.354520,0.923608,0.923608,-0.385002,0.000000,0.000000,0.397897,0.397897,-0.385002,0.923608,-0.385002,-0.385002,0.397897,0.000000,0.397897,-0.385002,-0.385002,0.000000,0.000000
2,0.0,-0.385002,-0.385002,1.354520,0.397897,1.354520,0.000000,0.000000,1.748038,-0.385002,0.397897,0.397897,-0.385002,-0.385002,-0.385002,0.397897,0.000000,-0.385002,0.923608,-0.385002
3,0.0,-0.385002,0.000000,0.000000,0.000000,0.397897,-0.385002,0.923608,-0.385002,0.923608,0.397897,0.000000,-0.385002,0.000000,0.000000,-0.385002,-0.385002,-0.385002,0.397897,0.000000
4,0.0,0.000000,-0.385002,0.000000,0.000000,-0.385002,-0.385002,0.397897,0.000000,0.923608,0.000000,0.397897,0.923608,0.397897,0.397897,-0.385002,-0.385002,0.923608,0.000000,0.397897


In [5]:
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_1_30per_testing_dataseat5000_Feb_2024.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.358999,0.039822,0.031013,0.200916,1.810602,-0.605655,1.410377,0.973277,0.272236,0.973277,-0.358999,-0.928147,0.000000,0.441866,0.242130,0.441866,-0.276281,-0.203610,-0.308443,0.201986
1,1.410377,-0.111723,-0.018455,0.200916,-0.462942,0.221536,0.000000,0.000000,-0.643134,0.000000,0.000000,-0.864143,-0.557091,0.326591,0.242130,0.326591,-0.249514,-0.077157,-0.308443,0.201986
2,-0.358999,-0.358999,0.004923,0.000000,-0.358999,-0.647981,0.000000,-0.358999,0.409120,-0.358999,-0.358999,0.172797,0.622058,0.289240,-0.284555,0.289240,-0.249514,0.480250,-0.308443,-0.872765
3,0.000000,0.900476,-0.018455,0.200916,-0.242985,0.517807,0.323957,-0.242985,0.683962,0.000000,0.000000,0.441866,0.323957,0.000000,0.242130,0.000000,-0.249514,-0.214692,-0.308443,0.201986
4,-0.358999,0.323957,-0.018455,0.200916,-0.358999,-0.013731,0.000000,-0.242985,0.000000,-0.557091,0.000000,-0.561457,-0.358999,0.000000,-0.284555,0.000000,-0.249514,-0.688589,-0.308443,0.201986


In [6]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, n_repeats=3, random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1270 23 94 113
pod:  0.5458937198067633
pof:  0.017788089713843776
AUC:  0.7640528150464597
recall:  0.5458937198067633
precision:  0.8308823529411765
F1-Score:  0.6588921282798834
accuracy:  0.922


In [7]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1292 1 136 71
pod:  0.34299516908212563
pof:  0.0007733952049497294
AUC:  0.6711108869385879
recall:  0.34299516908212563
precision:  0.9861111111111112
F1-Score:  0.5089605734767025
accuracy:  0.9086666666666666


In [8]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 183 24
pod:  0.11594202898550725
pof:  0.0007733952049497294
AUC:  0.5575843168902788
recall:  0.11594202898550725
precision:  0.96
F1-Score:  0.20689655172413793
accuracy:  0.8773333333333333


In [9]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 198 9
pod:  0.043478260869565216
pof:  0.0007733952049497294
AUC:  0.5213524328323077
recall:  0.043478260869565216
precision:  0.9
F1-Score:  0.08294930875576037
accuracy:  0.8673333333333333


In [10]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1262 31 136 71
pod:  0.34299516908212563
pof:  0.02397525135344161
AUC:  0.6595099588643419
recall:  0.34299516908212563
precision:  0.696078431372549
F1-Score:  0.45954692556634313
accuracy:  0.8886666666666667


In [11]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1291 2 126 81
pod:  0.391304347826087
pof:  0.0015467904098994587
AUC:  0.6948787787080938
recall:  0.391304347826087
precision:  0.9759036144578314
F1-Score:  0.5586206896551724
accuracy:  0.9146666666666666


In [12]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 195 12
pod:  0.057971014492753624
pof:  0.0
AUC:  0.5289855072463768
recall:  0.057971014492753624
precision:  1.0
F1-Score:  0.1095890410958904
accuracy:  0.87


In [13]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
88/88 [==============================] - 5s 10ms/step - loss: 0.3517 - accuracy: 0.8775 - val_loss: 0.1791 - val_accuracy: 0.9343
Epoch 2/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1937 - accuracy: 0.9275 - val_loss: 0.1267 - val_accuracy: 0.9600
Epoch 3/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1436 - accuracy: 0.9518 - val_loss: 0.1067 - val_accuracy: 0.9614
Epoch 4/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1263 - accuracy: 0.9550 - val_loss: 0.0919 - val_accuracy: 0.9729
Epoch 5/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1177 - accuracy: 0.9561 - val_loss: 0.0890 - val_accuracy: 0.9671
Epoch 6/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1095 - accuracy: 0.9593 - val_loss: 0.0869 - val_accuracy: 0.9686
Epoch 7/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1044 - accuracy: 0.9607 - val_loss: 0.0745 - val_accuracy: 0.9729
Epoch 8/30
88/88 [=

In [14]:
#RNN
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
max_features = 10000 # number of words to consider as features
#create model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(X_encoded, y_syn_df, epochs=5, batch_size=128, validation_split=0.2)
y2_pred = model.predict(X_encoded_test)
y22_pred=y2_pred.round()


from sklearn.metrics import confusion_matrix
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
22/22 [==============================] - 3s 27ms/step - loss: 0.6090 - accuracy: 0.8257 - val_loss: 0.4581 - val_accuracy: 0.8586
Epoch 2/5
22/22 [==============================] - 0s 6ms/step - loss: 0.3909 - accuracy: 0.8575 - val_loss: 0.3614 - val_accuracy: 0.858